In [1]:
import geopandas as gpd
import pandas as pd
import os
from shapely.geometry import Point, LineString
from busroute import snap_points_to_line, split_routes

# busroute.shp 的 欄位名稱(因每次資料來源提供的會有所不同)
route_routename_col = 'RouteNameZ'
route_direction_col = 'Direction'

# Seq 的欄位名稱 (因每次資料來源提供的會有所不同)
seq_routename_col = 'RouteName'
seq_direction_col = 'Direction'
seq_seq_col = 'Seq'
seq_lat_col = 'Lat'
seq_lng_col = 'Lon'

busroute = gpd.read_file(os.path.join(os.getcwd(),'..', 'input','Shp','BusRoute.shp'))
seq = seq = pd.read_csv(os.path.join(os.getcwd(),'..', 'input','seq.csv'))

In [2]:
# 只做同時有站序 & 路線檔案的
routelist = list(set(list(busroute[route_routename_col])) & set(list(seq[seq_routename_col])))
print("可計算的路線共有:", len(routelist),'條')
only_in_route = list(set(busroute[route_routename_col]) - set(seq[seq_routename_col]))
print("只有路線檔案的路線:", only_in_route)
only_in_seq = list(set(seq[seq_routename_col]) - set(busroute[route_routename_col]))
print("只有站序檔案的路線:", only_in_seq)

可計算的路線共有: 379 條
只有路線檔案的路線: []
只有站序檔案的路線: []


In [3]:
shp_outputfolder = os.path.join(os.getcwd(),'..', 'output','shp')
os.makedirs(shp_outputfolder, exist_ok=True)

以下嘗試

In [4]:
route = routelist[4]
print("現在示範的路線編號為：", route)

directions = [0,1]
direction = directions[0]
print("現在示範的方向為：", direction)

現在示範的路線編號為： L118
現在示範的方向為： 0


In [5]:
busroute_select = busroute[ (busroute[route_routename_col] == route) & (busroute[route_direction_col] == direction)][[route_routename_col,route_direction_col,'geometry' ]].reset_index(drop = True)
seq_select = seq[ (seq[seq_routename_col] == route) & (seq[seq_direction_col] == direction) ].sort_values(seq_seq_col).reset_index(drop = True)
# 假設 seq_select 包含 Lat 和 Lon 欄位，創建一個 geometry 欄位
seq_select['geometry'] = seq_select.apply(lambda row: Point(row[seq_lng_col], row[seq_lat_col]), axis=1)
# 將 Pandas DataFrame 轉換為 GeoDataFrame
seq_select = gpd.GeoDataFrame(seq_select, geometry='geometry').drop_duplicates(subset=[seq_seq_col]).reset_index(drop = True)
seq_select = seq_select.set_crs(epsg=4326, inplace=True)

In [107]:
# busroute_select.to_file(os.path.join(shp_outputfolder, 'busroute_select.shp'))
# seq_select.to_file(os.path.join(shp_outputfolder, 'seq_select.shp'))

In [6]:
# (1) 把公車站點（投影到路線上）
# 假設 busroute_select 和 seq_select 已經是 GeoDataFrames，並且你已經知道欄位名稱
seq_select = snap_points_to_line(seq_select, busroute_select, 
                                         route_routename_col=route_routename_col, 
                                         route_direction_col=route_direction_col, 
                                         seq_routename_col=seq_routename_col, 
                                         seq_direction_col=seq_direction_col, 
                                         seq_lat_col=seq_lat_col, 
                                         seq_lng_col=seq_lng_col)

(2) 把路線進行拆分

In [27]:
segment = split_routes(busroute_select, seq_select, route_routename_col = 'RouteNameZ').set_crs(epsg=4326, inplace=True)
segment = segment.to_crs(epsg=3826)
# 計算長度，這樣長度將以米為單位
segment['length'] = segment.geometry.length

segment['Direction'] = segment['Direction'].astype(int)
segment['OD'] = segment['StartSeq'].astype(str) + "-" + segment['EndSeq'].astype(str)
segment = segment[['RouteName', 'OD', 'Direction', 'StartSeq', 'EndSeq', 'length' ,'geometry']]


In [28]:
segment

,RouteName,OD,Direction,StartSeq,EndSeq,length,geometry
0,L118,1-2,0,1,2,346.696295,"LINESTRING (278403.503 2757570.066, 278404.137..."
1,L118,2-3,0,2,3,366.607347,"LINESTRING (278515.118 2757830.764, 278559.249..."
2,L118,3-4,0,3,4,321.231688,"LINESTRING (278741.162 2758024.177, 278741.887..."
3,L118,4-5,0,4,5,376.452089,"LINESTRING (278726.979 2758305.782, 278760.135..."
4,L118,5-6,0,5,6,476.002399,"LINESTRING (278763.202 2758678.049, 278772.721..."
...,...,...,...,...,...,...,...
65,L118,66-67,0,66,67,697.299248,"LINESTRING (277360.589 2758405.988, 277365.642..."
66,L118,67-68,0,67,68,952.683980,"LINESTRING (277990.654 2758209.756, 278091.163..."
67,L118,68-69,0,68,69,338.911731,"LINESTRING (278734.841 2758026.236, 278732.816..."
68,L118,69-70,0,69,70,275.436662,"LINESTRING (278730.454 2757687.842, 278730.488..."
